In [2]:
import pandas as pd
import json


ml_df = pd.read_csv('../data/ml_merged.csv')  # MovieLens: userId, movieId, rating, title
with open('../data/matched_amazon_reviews.jsonl') as f:
    matched_data = [json.loads(line) for line in f]
matched_df = pd.DataFrame(matched_data)  # movieId, asin

merged = pd.merge(ml_df, matched_df[['movieId', 'asin']], on='movieId', how='inner')

merged = merged.dropna(subset=['asin'])
print(f"Total data setelah merge: {len(merged)}")

Total data setelah merge: 109148727


In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

df_qi = pd.read_csv('../data/encoded_amazon_vectors.csv')
asin_order = df_qi['asin'].tolist()

item_encoder = LabelEncoder()
item_encoder.classes_ = np.array(asin_order)  
merged['item_index'] = item_encoder.transform(merged['asin'])

user_encoder = LabelEncoder()
merged['user_index'] = user_encoder.fit_transform(merged['userId'])


merged[['user_index', 'item_index', 'rating']].to_csv('../data/pmf_training_data.csv', index=False)

C:\Users\fikra\AppData\Local\Temp\ipykernel_20696\3639708975.py:4: DtypeWarning: Columns (256) have mixed types. Specify dtype option on import or set low_memory=False.
  df_qi = pd.read_csv('../data/encoded_amazon_vectors.csv')
